## HealtheIntent API's (including Personel Tool)
### Joe Eberle  

In [2]:
pip install Cerner-HealtheIntent

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install flatten_dict

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Import the healthintent api SO all personel fuunctions are available 
import healtheintentAPI as h 
from settings import *
import requests
import tarfile
import datetime as dt
import os
import pandas as pd 
from flatten_dict import flatten
from flatten_dict import unflatten

In [5]:
dir(h)

['BEARER_HEADER',
 'BEARER_HEADER_OP_API',
 'DataSyndication',
 'Healtheintent',
 'Organization',
 'Personnel',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'dt',
 'healtheintentAPI',
 'os',
 'requests',
 'tarfile']

In [7]:
o1 = h.Organization()
o = h.Organization()
foo=o.get_organization()
p = h.Personnel()

In [8]:
help(h.Organization)

Help on class Organization in module healtheintentAPI.healtheintentAPI:

class Organization(Healtheintent)
 |  # Organization Sub Class Commands
 |  
 |  Method resolution order:
 |      Organization
 |      Healtheintent
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  create_location(self, data=None, json=None)
 |  
 |  create_organization(self, data=None, json=None)
 |  
 |  delete_location(self, location_id=None)
 |  
 |  delete_organization(self, organization_id=None)
 |  
 |  get_location_status(self, location_id=None)
 |  
 |  get_locations(self)
 |      # Location Object Commands
 |  
 |  get_organization(self, params=None)
 |  
 |  update_location(self, location_id=None, data=None, json=None)
 |  
 |  update_organization(self, organization_id=None, data=None, json=None)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Healtheintent:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) fo

####  The following class defines all of the HealtheIntent Functions 

In [9]:
class Healtheintent(object):

    def __init__(self):

        self.uri = "https://wny.api.us.healtheintent.com/"
        self.headers = BEARER_HEADER
        self.headers_op = BEARER_HEADER_OP_API
        self.channel_id = None
        self.feed_id = None
        self.personnel_id = None
        self.organization_id = None
        self.personnel_group = None
        self.personnel_group_member_id = None
        self.location_id = None
        self.body = None

    def __send_request__(self, endpoint,
                         method = None, json = None, headers = None, params = None, data = None):
        if method == 'GET':
            return requests.get('{uri}{endpoint}'.format(uri=self.uri, endpoint=endpoint)
                                , headers=headers
                                , params=params).json()
        elif method == 'DELETE':
            return requests.delete('{uri}{endpoint}'.format(uri=self.uri, endpoint=endpoint)
                                   , headers=headers, params=params)
        elif method == 'POST':
            return requests.post('{uri}{endpoint}'.format(uri=self.uri, endpoint=endpoint),
                                   headers=headers, data=data, json=json)
        elif method == 'PUT':
            return requests.put('{uri}{endpoint}'.format(uri=self.uri, endpoint=endpoint),
                                   headers=headers, params=params,data=data,json=json)
        else:
            return requests.get('{uri}{endpoint}'.format(uri=self.uri, endpoint=endpoint)
                                , headers=headers, data=data, json=json)

#Data Syndication SubClass      
class DataSyndication(Healtheintent):

    def get_feeds(self):
        return self.__send_request__(endpoint='data-syndication/v1/feeds', method='GET', headers=self.headers)

    def get_channels(self):
        return self.__send_request__(endpoint='data-syndication/v1/channels', method='GET', headers=self.headers)

    def get_channel_status(self, channel_id = None):
        return self.__send_request__(endpoint='data-syndication/v1/channels/{channel_id}'.format(channel_id=channel_id),
                                     method='GET',
                                     headers=self.headers)

    def get_channel_deliveries(self, channel_id = None):
        return self.__send_request__(endpoint='data-syndication/v1/channels/{channel_id}/deliveries'.format(channel_id=channel_id),
                                     method='GET', headers=self.headers)

    def get_channel_downloads(self, delivery_id = None, destination_path = None):
        response = self.__send_request__(endpoint='data-syndication/v1/downloads/{delivery_id}'.format(delivery_id=delivery_id)
                                         , headers=self.headers)


        wd = os.getcwd()
        path = destination_path + '/' + dt.datetime.strftime(dt.datetime.today(), '%Y-%m-%d') + '.tar.gz'

        if response.status_code == 200:
            with open(path, 'wb') as i:
                i.write(response.content)
        i.close()

        temp = tarfile.open(name=path, mode='r:gz')
        os.chdir(destination_path)
        temp.extractall()
        temp.close()
        os.chdir(wd)


# Personnel Sub Class
class Personnel(Healtheintent):
    def get_personnel(self):
        return self.__send_request__(endpoint='personnel/v1/personnel', method='GET'
                                     , headers=self.headers_op)

    def create_personnel(self,data=None,json=None):
        return self.__send_request__(endpoint='personnel/v1/personnel'.format(data=data,json=json)
                                     , method='POST'
                                     , headers=self.headers_op
                                     , data=data
                                     , json=json)

    def get_personnel_status(self, personnel_id = None):
        return self.__send_request__(endpoint='personnel/v1/personnel/{personnel_id}'.format(personnel_id=personnel_id)
                                     , method='GET'
                                     , headers=self.headers_op)

    def update_personnel(self, personnel_id=None, data=None, json=None):
        return self.__send_request__(endpoint='personnel/v1/personnel/{personnel_id}'.format(personnel_id=personnel_id, data=data, json=json)
                                     , method='PUT', headers=self.headers_op, data=data, json=json)

    def delete_personnel(self, personnel_id = None):
        return self.__send_request__(endpoint='personnel/v1/personnel/{personnel_id}'.format(personnel_id=personnel_id)
                                     , method='DELETE'
                                     , headers=self.headers_op)

    
# Personnel Group Object
    def get_personnel_groups(self):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups'
                                     , method='GET'
                                     , headers=self.headers_op)

    def create_personnel_group(self, data=None, json=None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups'.format(data=data,json=json)
                                     , method='POST'
                                     , headers=self.headers_op)

    def get_personnel_group_status(self, personnel_group_id = None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups/{personnel_group_id}'.format(personnel_group_id=personnel_group_id)
                                     , method='GET'
                                     , headers=self.headers_op)

    def update_personnel_group(self, personnel_group_id = None, data=None, json=None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups/{personnel_group_id}'.format(personnel_group_id=personnel_group_id, data=data,json=json)
            , method='PUT'
            , headers=self.headers_op
            , data=data
            , json=json)

    def delete_personnel_group(self, personnel_group_id = None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups/{personnel_group_id}'.format(personnel_group_id=personnel_group_id)
                                     , method='DELETE'
                                     , headers=self.headers_op)

    def get_personnel_group_members(self, personnel_group_id = None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups/{personnel_group_id}/members'.format(personnel_group_id=personnel_group_id)
                                     , method='GET'
                                     , headers=self.headers_op)

    # Testing Required
    def add_personnel_group_members(self, personnel_group_id = None, personnel_group_member_id = None, data=None, json=None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups/{personnel_group_id}/members/{personnel_group_member_id}'
                                     , method='PUT'
                                     , headers=self.headers_op
                                    , data=data
                                    , json=json)

    def delete_personnel_group_members(self, personnel_group_id = None, personnel_group_member_id = None):
        return self.__send_request__(endpoint='personnel/v1/{personnel_group_id}/members/{personnel_group_member_id}'
                                     , method='DELETE'
                                     , headers=self.headers_op)


# Organization Sub Class Commands
class Organization(Healtheintent):
    
    def get_organization(self,params=None):
        return self.__send_request__(endpoint='organization/v1/organizations'.format(params=params)
                                     , method='GET'
                                     , headers=self.headers_op)
    
    def create_organization(self,data=None,json=None):
        return self.__send_request__(endpoint='organization/v1/organizations'.format(data=data, json=json)
                                     , method='POST'
                                     , headers=self.headers_op
                                     , data=data
                                     , json=json)
    
    def delete_organization(self, organization_id = None):
        return self.__send_request__(endpoint='organization/v1/organizations/{organization_id}'.format(organization_id=organization_id)
                                     , method='DELETE'
                                     , headers=self.headers_op)
    
    def update_organization(self, organization_id = None, data=None, json=None):
        return self.__send_request__(endpoint='organization/v1/organizations/{organization_id}'.format(organization_id=organization_id,data=data,json=json)
                                     , method='PUT'
                                     , headers=self.headers_op
                                    , data=data
                                    , json=json)
    
    
# Organization Sub Class Commands
class Organization_Group(Healtheintent):
    
    def get_organization_group(self,params=None):
        return self.__send_request__(endpoint='organization/v1/organizations'.format(params=params)
                                     , method='GET'
                                     , headers=self.headers_op)
    
    def create_organization_group(self,data=None,json=None):
        return self.__send_request__(endpoint='organization/v1/organizations'.format(data=data, json=json)
                                     , method='POST'
                                     , headers=self.headers_op
                                     , data=data
                                     , json=json)
    
    def delete_organization_group(self, organization_id = None):
        return self.__send_request__(endpoint='organization/v1/organizations/{organization_id}'.format(organization_id=organization_id)
                                     , method='DELETE'
                                     , headers=self.headers_op)
    
    def update_organization_group(self, organization_id = None, data=None, json=None):
        return self.__send_request__(endpoint='organization/v1/organizations/{organization_id}'.format(organization_id=organization_id,data=data,json=json)
                                     , method='PUT'
                                     , headers=self.headers_op
                                    , data=data
                                    , json=json)    
    
    
    
    
    # Location Object Commands
    def get_locations(self):
        return self.__send_request__(endpoint='organization/v1/locations'
                                     , method='GET'
                                     , headers=self.headers_op)

    def create_location(self,data=None,json=None):
        return self.__send_request__(endpoint='organization/v1/locations'.format(data=data, json=json)
                                     , method='POST'
                                     , headers=self.headers_op
                                     , data=data
                                     , json=json)
    
    def get_location_status(self, location_id = None):
        return self.__send_request__(endpoint='organization/v1/locations/{location_id}'.format(location_id=location_id)
                                     , method='GET'
                                     , headers=self.headers_op)

    def update_location(self, location_id = None, data=None, json= None):
        return self.__send_request__(endpoint='organization/v1/locations/{location_id}'.format(location_id=location_id,data=data,json=json)
                                     , method='PUT'
                                     , headers=self.headers_op
                                    , data=data
                                    , json=json)

    def delete_location(self, location_id = None):
        return self.__send_request__(endpoint='organization/v1/locations/{location_id}'.format(location_id=location_id)
                                     , method='DELETE'
                                     , headers=self.headers_op)
    
    

In [10]:
help(Healtheintent)


Help on class Healtheintent in module __main__:

class Healtheintent(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __send_request__(self, endpoint, method=None, json=None, headers=None, params=None, data=None)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [26]:
# POST Testing - Update a Clinic 
# import json
org_json_template_example = {"name":"Joes Test Clinic",
        "coverageAreaPostalCodes":['14216'],
        "telecoms":[{'system':'PHONE',
                    'value':'7198675209'}],
        "addresses":[{"postalCode":'14216', 
                      "city":'Buffalo',
                      "state":'NY',
                      'lines':['680','Seneca St']
                     }],
         "aliases":[{"type":'SOI',
                     "value":'0123456789',
                     "system":'2.16.840.1.113883.4.6'}],
        'sourceIdentifiers': []}



In [24]:
print(org_json_template_example) 

{'name': 'Joes Test Clinic', 'coverageAreaPostalCodes': ['14216'], 'telecoms': [{'system': 'PHONE', 'value': '7198675209'}], 'addresses': [{'postalCode': '14216', 'city': 'Buffalo', 'state': 'NY', 'lines': ['680', 'Seneca St']}], 'aliases': [{'type': 'SOI', 'value': '0123456789', 'system': '2.16.840.1.113883.4.6'}], 'sourceIdentifiers': []}


In [27]:
#body=json.dumps(body3)
o.create_organization(json=org_json_template_example).json()

{'id': '64c43eb7-dab0-41ad-9780-e25fefa2d1c8',
 'name': 'Joes Test Clinic',
 'telecoms': [{'system': 'PHONE', 'value': '7198675209'}],
 'addresses': [{'postalCode': '14216',
   'city': 'Buffalo',
   'state': 'NY',
   'lines': ['680', 'Seneca St']}],
 'aliases': [{'type': 'SOI',
   'value': '0123456789',
   'system': '2.16.840.1.113883.4.6'}],
 'sourceIdentifiers': [],
 'coverageAreaPostalCodes': ['14216']}

In [29]:
o.delete_organization('64c43eb7-dab0-41ad-9780-e25fefa2d1c8') 

<Response [204]>

In [30]:
# POST Testing - Update a Clinic 
# import json
org_json = {"name":"GPPC PCP Amherst Sheridan",
        "coverageAreaPostalCodes":['14226'],
        "telecoms":[{'system':'PHONE',
                    'value':'7168332200'}],
        "addresses":[{"postalCode":'14226', 
                      "city":'Amherst',
                      "state":'NY',
                      'lines':['3980A Sheridan Drive','Suite 200']
                     }],
         "aliases":[{"type":'SOI',
                     "value":'1093959322',
                     "system":'2.16.840.1.113883.4.6'},
                    {"type":'TAX',
                     "value":'161460136',
                     "system":'2.16.840.1.113883.4.6'}],
        'sourceIdentifiers': []}
#body=json.dumps(body3)
o.create_organization(json=org_json).json()


{'code': 409,
 'message': 'Conflict',
 'errorDetails': [{'message': 'One or more aliases in the entity body are already associated with existing organizations.',
   'location': 'aliases',
   'reason': 'organizationWithAliasAlreadyExists'}]}

In [31]:
# POST Testing - Update a Clinic 'id': '7fcd36af-19cb-4de4-ae76-ac34181175eb',
# import json
org_json = {"name":"GPPC PCP Amherst Sheridan",
        "coverageAreaPostalCodes":['14226'],
        "telecoms":[{'system':'PHONE',
                    'value':'7168332200'}],
        "addresses":[{"postalCode":'14226', 
                      "city":'Amherst',
                      "state":'NY',
                      'lines':['3980A Sheridan Drive','Suite 200']
                     }],
        'sourceIdentifiers': []}
#body=json.dumps(body3)
o.create_organization(json=org_json).json()

{'id': '7fcd36af-19cb-4de4-ae76-ac34181175eb',
 'name': 'GPPC PCP Amherst Sheridan',
 'telecoms': [{'system': 'PHONE', 'value': '7168332200'}],
 'addresses': [{'postalCode': '14226',
   'city': 'Amherst',
   'state': 'NY',
   'lines': ['3980A Sheridan Drive', 'Suite 200']}],
 'aliases': [],
 'sourceIdentifiers': [],
 'coverageAreaPostalCodes': ['14226']}

In [32]:
# POST Testing - Update a Clinic  {'id': '5811b42a-aabb-4387-9bfd-6757b830adde',
# import json
org_json = {"name":"GPPC PCP Buffalo Main",
        "coverageAreaPostalCodes":['14209'],
        "telecoms":[{'system':'PHONE',
                    'value':'7168524772'}],
        "addresses":[{"postalCode":'14209', 
                      "city":'Buffalo',
                      "state":'NY',
                      'lines':['1091 Main St','Ste 301']
                     }],
        'sourceIdentifiers': []}
#body=json.dumps(body3)
o.create_organization(json=org_json).json()

{'id': '5811b42a-aabb-4387-9bfd-6757b830adde',
 'name': 'GPPC PCP Buffalo Main',
 'telecoms': [{'system': 'PHONE', 'value': '7168524772'}],
 'addresses': [{'postalCode': '14209',
   'city': 'Buffalo',
   'state': 'NY',
   'lines': ['1091 Main St', 'Ste 301']}],
 'aliases': [],
 'sourceIdentifiers': [],
 'coverageAreaPostalCodes': ['14209']}

In [33]:
# POST Testing - Update a Clinic  {'id': '31f310f6-9330-4987-a2b1-c9ba32386e90',
# import json
org_json = {"name":"GPPC PCP Dunkirk Vineyard Ste 1",
        "coverageAreaPostalCodes":['14048'],
        "telecoms":[{'system':'PHONE',
                    'value':'7163636960'}],
        "addresses":[{"postalCode":'14048', 
                      "city":'Dunkirk',
                      "state":'NY',
                      'lines':['3898 Vineyard Dr','Ste 1']
                     }],
        'sourceIdentifiers': []}
#body=json.dumps(body3)
o.create_organization(json=org_json).json()

{'id': '31f310f6-9330-4987-a2b1-c9ba32386e90',
 'name': 'GPPC PCP Dunkirk Vineyard Ste 1',
 'telecoms': [{'system': 'PHONE', 'value': '7163636960'}],
 'addresses': [{'postalCode': '14048',
   'city': 'Dunkirk',
   'state': 'NY',
   'lines': ['3898 Vineyard Dr', 'Ste 1']}],
 'aliases': [],
 'sourceIdentifiers': [],
 'coverageAreaPostalCodes': ['14048']}

In [34]:
# POST Testing - Update a Clinic  {'id': 'a696e667-2027-4430-91f1-e5e1d763c509',
# import json
org_json = {"name":"GPPC PCP East Aurora Main",
        "coverageAreaPostalCodes":['14052'],
        "telecoms":[{'system':'PHONE',
                    'value':'7165394200'}],
        "addresses":[{"postalCode":'14052', 
                      "city":'E Aurora',
                      "state":'NY',
                      'lines':['268 Main St',' ']
                     }],
        'sourceIdentifiers': []}
#body=json.dumps(body3)
o.create_organization(json=org_json).json()

{'id': 'a696e667-2027-4430-91f1-e5e1d763c509',
 'name': 'GPPC PCP East Aurora Main',
 'telecoms': [{'system': 'PHONE', 'value': '7165394200'}],
 'addresses': [{'postalCode': '14052',
   'city': 'E Aurora',
   'state': 'NY',
   'lines': ['268 Main St']}],
 'aliases': [],
 'sourceIdentifiers': [],
 'coverageAreaPostalCodes': ['14052']}

In [35]:
# POST Testing - Update a Clinic  {'id': '9aade2bb-4162-4026-9263-4ab98c372b4b',
org_json = {"name":"GPPC PCP Elma Transit",
        "coverageAreaPostalCodes":['14059'],
        "telecoms":[{'system':'PHONE',
                    'value':'7166265840'}],
        "addresses":[{"postalCode":'14059', 
                      "city":'Elma',
                      "state":'NY',
                      'lines':['2701 Transit Rd','Ste 143']
                     }],
        'sourceIdentifiers': []}
o.create_organization(json=org_json).json()

{'id': '9aade2bb-4162-4026-9263-4ab98c372b4b',
 'name': 'GPPC PCP Elma Transit',
 'telecoms': [{'system': 'PHONE', 'value': '7166265840'}],
 'addresses': [{'postalCode': '14059',
   'city': 'Elma',
   'state': 'NY',
   'lines': ['2701 Transit Rd', 'Ste 143']}],
 'aliases': [],
 'sourceIdentifiers': [],
 'coverageAreaPostalCodes': ['14059']}

In [36]:
# POST Testing - Update a Clinic  {'id': '288e25e9-287a-4814-aebe-1563a58b1c34',
org_json = {"name":"GPPC PCP Hamburg Camp",
        "coverageAreaPostalCodes":['14075'],
        "telecoms":[{'system':'PHONE',
                    'value':'7166462590'}],
        "addresses":[{"postalCode":'14075', 
                      "city":'Hamburg',
                      "state":'NY',
                      'lines':['5470 Camp Rd','Ste 200']
                     }],
        'sourceIdentifiers': []}
o.create_organization(json=org_json).json()

{'id': '288e25e9-287a-4814-aebe-1563a58b1c34',
 'name': 'GPPC PCP Hamburg Camp',
 'telecoms': [{'system': 'PHONE', 'value': '7166462590'}],
 'addresses': [{'postalCode': '14075',
   'city': 'Hamburg',
   'state': 'NY',
   'lines': ['5470 Camp Rd', 'Ste 200']}],
 'aliases': [],
 'sourceIdentifiers': [],
 'coverageAreaPostalCodes': ['14075']}

In [37]:
# POST Testing - Update a Clinic  {'id': '0112276c-9776-40ab-b1d4-4ee44cb219db',
org_json = {"name":"GPPC PCP Hamburg Southwestern",
        "coverageAreaPostalCodes":['14075'],
        "telecoms":[{'system':'PHONE',
                    'value':'7167108266'}],
        "addresses":[{"postalCode":'14075', 
                      "city":'Hamburg',
                      "state":'NY',
                      'lines':['5844 Southwestern Blvd','Ste 500']
                     }],
        'sourceIdentifiers': []}
o.create_organization(json=org_json).json()

{'id': '0112276c-9776-40ab-b1d4-4ee44cb219db',
 'name': 'GPPC PCP Hamburg Southwestern',
 'telecoms': [{'system': 'PHONE', 'value': '7167108266'}],
 'addresses': [{'postalCode': '14075',
   'city': 'Hamburg',
   'state': 'NY',
   'lines': ['5844 Southwestern Blvd', 'Ste 500']}],
 'aliases': [],
 'sourceIdentifiers': [],
 'coverageAreaPostalCodes': ['14075']}

In [38]:
# POST Testing - Update a Clinic  {'id': 'de44fcf4-6fea-4418-9d31-7fc094f6cdc1'
org_json = {"name":"GPPC PCP Lancaster Transit",
        "coverageAreaPostalCodes":['14043'],
        "telecoms":[{'system':'PHONE',
                    'value':'7166087040'}],
        "addresses":[{"postalCode":'14043', 
                      "city":'Depew',
                      "state":'NY',
                      'lines':['4893 Transit Rd','']
                     }],
        'sourceIdentifiers': []}
o.create_organization(json=org_json).json()

{'id': 'de44fcf4-6fea-4418-9d31-7fc094f6cdc1',
 'name': 'GPPC PCP Lancaster Transit',
 'telecoms': [{'system': 'PHONE', 'value': '7166087040'}],
 'addresses': [{'postalCode': '14043',
   'city': 'Depew',
   'state': 'NY',
   'lines': ['4893 Transit Rd']}],
 'aliases': [],
 'sourceIdentifiers': [],
 'coverageAreaPostalCodes': ['14043']}

In [39]:
# POST Testing - Update a Clinic  'id': 'e340fef4-c36b-449b-8f3d-4f40d658bfeb',
org_json = {"name":"GPPC PCP Lockport Olcott",
        "coverageAreaPostalCodes":['14094'],
        "telecoms":[{'system':'PHONE',
                    'value':'7168524772'}],
        "addresses":[{"postalCode":'14094', 
                      "city":'Lockport',
                      "state":'NY',
                      'lines':['3805 Lockport-Olcott Rd','']
                     }],
        'sourceIdentifiers': []}
o.create_organization(json=org_json).json()

{'id': 'e340fef4-c36b-449b-8f3d-4f40d658bfeb',
 'name': 'GPPC PCP Lockport Olcott',
 'telecoms': [{'system': 'PHONE', 'value': '7168524772'}],
 'addresses': [{'postalCode': '14094',
   'city': 'Lockport',
   'state': 'NY',
   'lines': ['3805 Lockport-Olcott Rd']}],
 'aliases': [],
 'sourceIdentifiers': [],
 'coverageAreaPostalCodes': ['14094']}

In [40]:
# POST Testing - Update a Clinic  {'id': 'be514be9-c337-400a-9996-e822650f773d',
org_json = {"name":"GPPC PCP Lockport Snyder",
        "coverageAreaPostalCodes":['14094'],
        "telecoms":[{'system':'PHONE',
                    'value':'7164338751'}],
        "addresses":[{"postalCode":'14094', 
                      "city":'Lockport',
                      "state":'NY',
                      'lines':['5879 Snyder Rd','']
                     }],
        'sourceIdentifiers': []}
o.create_organization(json=org_json).json()

{'id': 'be514be9-c337-400a-9996-e822650f773d',
 'name': 'GPPC PCP Lockport Snyder',
 'telecoms': [{'system': 'PHONE', 'value': '7164338751'}],
 'addresses': [{'postalCode': '14094',
   'city': 'Lockport',
   'state': 'NY',
   'lines': ['5879 Snyder Rd']}],
 'aliases': [],
 'sourceIdentifiers': [],
 'coverageAreaPostalCodes': ['14094']}

In [41]:
# POST Testing - Update a Clinic  {'id': 'be514be9-c337-400a-9996-e822650f773d',
org_json = {"name":"GPPC PCP Medina Ohio",
        "coverageAreaPostalCodes":['14103'],
        "telecoms":[{'system':'PHONE',
                    'value':'5857983345'}],
        "addresses":[{"postalCode":'14103', 
                      "city":'Medina',
                      "state":'NY',
                      'lines':['100 Ohio St.','']
                     }],
        'sourceIdentifiers': []}
o.create_organization(json=org_json).json()

{'id': '6bb543e1-9f6b-4cb0-a259-f383b83d6c9f',
 'name': 'GPPC PCP Medina Ohio',
 'telecoms': [{'system': 'PHONE', 'value': '5857983345'}],
 'addresses': [{'postalCode': '14103',
   'city': 'Medina',
   'state': 'NY',
   'lines': ['100 Ohio St.']}],
 'aliases': [],
 'sourceIdentifiers': [],
 'coverageAreaPostalCodes': ['14103']}

In [42]:
# POST Testing - Update a Clinic  {'id': '8de2a8fe-d8e3-43b3-a8b8-b243e36d0904',
org_json = {"name":"GPPC PCP Orchard Park Big Tree Ste 107",
        "coverageAreaPostalCodes":['14127'],
        "telecoms":[{'system':'PHONE',
                    'value':'7168524772'}],
        "addresses":[{"postalCode":'14127', 
                      "city":'Orchard Park',
                      "state":'NY',
                      'lines':['5959 Big Tree Rd','Ste 107']
                     }],
        'sourceIdentifiers': []}
o.create_organization(json=org_json).json()

{'id': '8de2a8fe-d8e3-43b3-a8b8-b243e36d0904',
 'name': 'GPPC PCP Orchard Park Big Tree Ste 107',
 'telecoms': [{'system': 'PHONE', 'value': '7168524772'}],
 'addresses': [{'postalCode': '14127',
   'city': 'Orchard Park',
   'state': 'NY',
   'lines': ['5959 Big Tree Rd', 'Ste 107']}],
 'aliases': [],
 'sourceIdentifiers': [],
 'coverageAreaPostalCodes': ['14127']}

In [44]:
# POST Testing - Update a Clinic  {'id': '3d130551-30d5-463e-be1a-467526866205',
org_json = {"name":"GPPC PCP Springville South Cascade",
        "coverageAreaPostalCodes":['14141'],
        "telecoms":[{'system':'PHONE',
                    'value':'7165923600'}],
        "addresses":[{"postalCode":'14141', 
                      "city":'Springville',
                      "state":'NY',
                      'lines':['230 S. Cascade Dr','']
                     }],
        'sourceIdentifiers': []}
o.create_organization(json=org_json).json()

{'id': '3d130551-30d5-463e-be1a-467526866205',
 'name': 'GPPC PCP Springville South Cascade',
 'telecoms': [{'system': 'PHONE', 'value': '7165923600'}],
 'addresses': [{'postalCode': '14141',
   'city': 'Springville',
   'state': 'NY',
   'lines': ['230 S. Cascade Dr']}],
 'aliases': [],
 'sourceIdentifiers': [],
 'coverageAreaPostalCodes': ['14141']}

In [45]:
# POST Testing - Update a Clinic  {'id': '43eeac13-81ac-47ea-9df0-066693fb736d',
org_json = {"name":"GPPC PCP Tonawanda Colvin",
        "coverageAreaPostalCodes":['14223'],
        "telecoms":[{'system':'PHONE',
                    'value':'7168744060'}],
        "addresses":[{"postalCode":'14223', 
                      "city":'Tonawanda',
                      "state":'NY',
                      'lines':['1430 Colvin Blvd','']
                     }],
        'sourceIdentifiers': []}
o.create_organization(json=org_json).json()

{'id': '43eeac13-81ac-47ea-9df0-066693fb736d',
 'name': 'GPPC PCP Tonawanda Colvin',
 'telecoms': [{'system': 'PHONE', 'value': '7168744060'}],
 'addresses': [{'postalCode': '14223',
   'city': 'Tonawanda',
   'state': 'NY',
   'lines': ['1430 Colvin Blvd']}],
 'aliases': [],
 'sourceIdentifiers': [],
 'coverageAreaPostalCodes': ['14223']}

In [46]:
# POST Testing - Update a Clinic  {'id': 'bd79cbe4-e0e7-4647-9f2e-44e57e4ca0b4',
org_json = {"name":"GPPC PCP Willamsville Maple",
        "coverageAreaPostalCodes":['14221'],
        "telecoms":[{'system':'PHONE',
                    'value':'7167108266'}],
        "addresses":[{"postalCode":'14221', 
                      "city":'Williamsville',
                      "state":'NY',
                      'lines':['705 Maple Rd','Ste 300']
                     }],
        'sourceIdentifiers': []}
o.create_organization(json=org_json).json()

{'id': 'bd79cbe4-e0e7-4647-9f2e-44e57e4ca0b4',
 'name': 'GPPC PCP Willamsville Maple',
 'telecoms': [{'system': 'PHONE', 'value': '7167108266'}],
 'addresses': [{'postalCode': '14221',
   'city': 'Williamsville',
   'state': 'NY',
   'lines': ['705 Maple Rd', 'Ste 300']}],
 'aliases': [],
 'sourceIdentifiers': [],
 'coverageAreaPostalCodes': ['14221']}

In [47]:
# POST Testing - Update a Clinic  {'id': 'bd79cbe4-e0e7-4647-9f2e-44e57e4ca0b4',
org_json = {"name":"GPPC PCP Williamsville Youngs",
        "coverageAreaPostalCodes":['14221'],
        "telecoms":[{'system':'PHONE',
                    'value':'7169327777'}],
        "addresses":[{"postalCode":'14221', 
                      "city":'Williamsville',
                      "state":'NY',
                      'lines':['1000 Youngs Rd','']
                     }],
        'sourceIdentifiers': []}
o.create_organization(json=org_json).json()

{'id': 'c2553d46-35f7-4a09-8de9-51cc07364dc9',
 'name': 'GPPC PCP Williamsville Youngs',
 'telecoms': [{'system': 'PHONE', 'value': '7169327777'}],
 'addresses': [{'postalCode': '14221',
   'city': 'Williamsville',
   'state': 'NY',
   'lines': ['1000 Youngs Rd']}],
 'aliases': [],
 'sourceIdentifiers': [],
 'coverageAreaPostalCodes': ['14221']}

In [49]:
# POST Testing - Update a Clinic  {'id': '586d6a46-55a7-4b56-b691-c3988062057f',
org_json = {"name":"GPPC Peds Dunkirk Vineyard Ste 3",
        "coverageAreaPostalCodes":['14048'],
        "telecoms":[{'system':'PHONE',
                    'value':'7163636960'}],
        "addresses":[{"postalCode":'14048', 
                      "city":'Williamsville',
                      "state":'NY',
                      'lines':['3898 Vineyard Drive','Ste 3']
                     }],
        'sourceIdentifiers': []}
o.create_organization(json=org_json).json()



{'id': '586d6a46-55a7-4b56-b691-c3988062057f',
 'name': 'GPPC Peds Dunkirk Vineyard Ste 3',
 'telecoms': [{'system': 'PHONE', 'value': '7163636960'}],
 'addresses': [{'postalCode': '14048',
   'city': 'Williamsville',
   'state': 'NY',
   'lines': ['3898 Vineyard Drive', 'Ste 3']}],
 'aliases': [],
 'sourceIdentifiers': [],
 'coverageAreaPostalCodes': ['14048']}

In [50]:
# POST Testing - Update a Clinic  {'id': '586d6a46-55a7-4b56-b691-c3988062057f',
org_json = {"name":"GPPC Post-Accute",
        "coverageAreaPostalCodes":['14215'],
        "telecoms":[{'system':'PHONE',
                    'value':'7163094772'}],
        "addresses":[{"postalCode":'14215', 
                      "city":'Buffalo',
                      "state":'NY',
                      'lines':['462 Grider Street','Terrace View at ECMC']
                     }],
        'sourceIdentifiers': []}
o.create_organization(json=org_json).json()


{'id': 'bbd76129-76f7-41dd-b569-4ad7cf9f9165',
 'name': 'GPPC Post-Accute',
 'telecoms': [{'system': 'PHONE', 'value': '7163094772'}],
 'addresses': [{'postalCode': '14215',
   'city': 'Buffalo',
   'state': 'NY',
   'lines': ['462 Grider Street', 'Terrace View at ECMC']}],
 'aliases': [],
 'sourceIdentifiers': [],
 'coverageAreaPostalCodes': ['14215']}

In [ ]:
GPPC PCP Amherst Sheridan	3980A Sheridan Dr, Ste 200, Amherst, 14226	(716)833-2200
GPPC PCP Buffalo Main	1091 Main St, Ste 301, Buffalo, NY 14209	(716)852-4772
GPPC PCP Dunkirk Vineyard Ste 1	3898 Vineyard Dr, Ste 1, Dunkirk, NY 14048	(716)363-6960
GPPC PCP East Aurora Main	268 Main St, E Aurora, 14052	(716)539-4200
GPPC PCP Elma Transit	2701 Transit Rd, Ste 143, Elma, NY 14059	(716)626-5840
GPPC PCP Hamburg Camp	5470 Camp Rd, Ste 200, Hamburg, NY 14075	(716)646-2590
GPPC PCP Hamburg Southwestern	5844 Southwestern Blvd, Ste 500, Hamburg, NY 14075	(716)710-8266
GPPC PCP Lancaster Transit	4893 Transit Rd, Depew, 14043	(716)608-7040
GPPC PCP Lockport Olcott	3805 Lockport-Olcott Rd, Lockport, 14094	(716)852-4772
GPPC PCP Lockport Snyder	5879 Snyder Rd, Lockport, 14094	(716)433-8751
GPPC PCP Medina Ohio	100 Ohio St., Medina, NY 14103	(585)798-3345
GPPC PCP Orchard Park Big Tree Ste 107	5959 Big Tree Rd, Ste 107, Orchard Park, NY 14127	(716)852-4772
GPPC PCP Springville South Cascade	230 S. Cascade Dr, Springville, NY 14141	(716)592-3600
GPPC PCP Tonawanda Colvin	1430 Colvin Blvd, Tonawanda, NY 14223	(716)874-4060
GPPC PCP Willamsville Maple	705 Maple Rd, Ste 300, Williamsville, NY 14221	(716)710-8266
GPPC PCP Williamsville Youngs	1000 Youngs Rd, Williamsville, NY 14221	(716)932-7777
GPPC Peds Dunkirk Vineyard Ste 3	3898 Vineyard Drive, Suite 3, Dunkirk, NY 14048	(716)363-6960


In [ ]:
o.get_organization(json=org_json_template).json()

In [ ]:
# POST Testing - Update a Clinic 
# import json
org_json_template_example = {"name":"GPPC PCP Buffalo Main",
        "coverageAreaPostalCodes":['14216'],
        "telecoms":[{'system':'PHONE',`
                    'value':'7198675209'}],
        "addresses":[{"postalCode":'14216', 
                      "city":'Buffalo',
                      "state":'NY',
                      'lines':['680','Seneca St']
                     }],
         "aliases":[{"type":'SOI',
                     "value":'0123456789',
                     "system":'2.16.840.1.113883.4.6'}
                    {"type":'TAX',
                     "value":'0123456789',
                     "system":'2.16.840.1.113883.4.6'}
                   ],
        'sourceIdentifiers': []}
#body=json.dumps(body3)
o.create_organization(json=org_json_template).json()


In [ ]:
# POST Testing
# import json 
org_json_template = {"name":"Fld1_Org_Name",
        "coverageAreaPostalCodes":['Fld2_Coverage_Postal'],
        "telecoms":[{'system':'Fld3_telecoms_phone_label',
                    'value':'Fld4_telecoms_phone'}],
        "addresses":[{"postalCode":'Fld5_addresses_postalCode', 
                      "city":'Fld6_addresses_City',
                      "state":'Fld7_addresses_State',
                      'lines':['Fld8_addresses_Line1','Fld9_addresses_Line2']
                     }],
         "aliases":[{"type":'Fld10_aliases_type',
                     "value":'Fld11_aliases_value',
                     "system":'Fld12_aliases_system'}],
        'sourceIdentifiers': []}

body=org_json_template.dumps()
#o.create_organization(json=org_json_template).json()

In [ ]:
dir(h)


In [14]:
org_json_template = {"name":"Fld1_Org_Name",
        "coverageAreaPostalCodes":['Fld2_Coverage_Postal'],
        "telecoms":[{'system':'Fld3_telecoms_phone_label',
                    'value':'Fld4_telecoms_phone'}],
        "addresses":[{"postalCode":'Fld5_addresses_postalCode', 
                      "city":'Fld6_addresses_City',
                      "state":'Fld7_addresses_State',
                      'lines':['Fld8_addresses_Line1','Fld9_addresses_Line2']
                     }],
         "aliases":[{"type":'Fld10_aliases_type',
                     "value":'Fld11_aliases_value',
                     "system":'Fld12_aliases_system'}],
        'sourceIdentifiers': []}

org_json_template["name"] = 'ORG NAME'
#org_json_template['telecoms']['system'] =  'blah' 

print(org_json_template) 


{'name': 'ORG NAME', 'coverageAreaPostalCodes': ['Fld2_Coverage_Postal'], 'telecoms': [{'system': 'Fld3_telecoms_phone_label', 'value': 'Fld4_telecoms_phone'}], 'addresses': [{'postalCode': 'Fld5_addresses_postalCode', 'city': 'Fld6_addresses_City', 'state': 'Fld7_addresses_State', 'lines': ['Fld8_addresses_Line1', 'Fld9_addresses_Line2']}], 'aliases': [{'type': 'Fld10_aliases_type', 'value': 'Fld11_aliases_value', 'system': 'Fld12_aliases_system'}], 'sourceIdentifiers': []}


In [ ]:
def fill_org_template(Fld1_Org_Name, Fld2_Coverage_Postal, Fld3_telecoms_phone_label
                      , Fld4_telecoms_phone,Fld5_addresses_postalCode,Fld6_addresses_City
                      , Fld7_addresses_State,Fld8_addresses_Line1,Fld9_addresses_Line2
                      , Fld10_aliases_type,Fld11_aliases_value,Fld12_aliases_system):

    org_json_template = {"name":"Fld1_Org_Name",
        "coverageAreaPostalCodes":['Fld2_Coverage_Postal'],
        "telecoms":[{'system':'Fld3_telecoms_phone_label',
                    'value':'Fld4_telecoms_phone'}],
        "addresses":[{"postalCode":'Fld5_addresses_postalCode', 
                      "city":'Fld6_addresses_City',
                      "state":'Fld7_addresses_State',
                      'lines':['Fld8_addresses_Line1','Fld9_addresses_Line2']
                     }],
         "aliases":[{"type":'Fld10_aliases_type',
                     "value":'Fld11_aliases_value',
                     "system":'Fld12_aliases_system'}],
        'sourceIdentifiers': []}
    org_json_template
    org_json_template.update({"name": Fld1_Org_Name})
    org_json_template.update({"coverageAreaPostalCodes": Fld2_Coverage_Postal})
    org_json_template.update({"system": Fld3_telecoms_phone_label})
    org_json_template.update({"value": Fld4_telecoms_phone})
    org_json_template.update({"postalCode": Fld5_addresses_postalCode})
    org_json_template.update({"city": Fld6_addresses_City})
    org_json_template.update({"state": Fld7_addresses_State})
    org_json_template.update({"lines": Fld8_addresses_Line1})
    org_json_template.update({"type": Fld10_aliases_type})
    org_json_template.update({"value": Fld11_aliases_value})    
    org_json_template.update({"system": Fld12_aliases_system})     
    
    return org_json_template



In [ ]:
org = fill_org_template('fld1','fld2','fld3','fld4','fld5','fld6','fld7','fld8','fld9','fld10','fld11','fld12')
print(org1)

In [ ]:
org2 =  flatten(org1)
print(org2)

In [ ]:
def dict_flatten(in_dict, dict_out=None, parent_key=None, separator="_"):
   if dict_out is None:
      dict_out = {}

   for k, v in in_dict.items():
      k = f"{parent_key}{separator}{k}" if parent_key else k
      if isinstance(v, dict):
         dict_flatten(in_dict=v, dict_out=dict_out, parent_key=k)
         continue

      dict_out[k] = v

   return dict_out



In [ ]:
org = fill_org_template('Joes Test Clinic','14226','PHONE','(716)999-9999','14225','Buffalo','NY','680 Seneca Street','Address line 2 ??','SPI','123456789','55555')
print(org)

In [ ]:
# POST Testing
#import json
temp = {"name":"Joes Test Clinic",
        "coverageAreaPostalCodes":['14216'],
        "telecoms":[{'system':'PHONE',
                    'value':'7198675209'}],
        "addresses":[{"postalCode":'14216', 
                      "city":'Buffalo',
                      "state":'NY',
                      'lines':['680','Seneca St']
                     }],
#         "aliases":[{"type":'SOI',
#                     "value":'0123456789',
#                     "system":'2.16.840.1.113883.4.6'}],
        'sourceIdentifiers': []}
#body=json.dumps(body3)
o.create_organization(json=temp).json()

In [ ]:
temp = {"birthDate": "1970-11-15",
      "gender": "MALE",
      "name": {
        "prefix": "Dr.",
        "given": "Joseph",
        "middle": "Test",
        "family": "Eberle",
     },
      "aliases": [
        {
          "value": "1234567890",
          "system": "5ecaf544-01d5-01kf-95hj-8e2bcec12006",
          "type": "SPI"
        }
      ],
      "telecoms": [
        {
          "system": "PHONE",
          "value": "111-111-1111"
        },
        {
          "system": "EMAIL",
          "value": "jane.smith@cerner.com"
        }
            ]
}

#body=json.dumps(body3)
p.create_personnel(json=temp).json()

In [ ]:
#body=json.dumps(body3)
o.create_organization(json=temp).json()

In [ ]:
# POST Testing
#import json
temp = {"name":"Joe Test Provider ",
        "coverageAreaPostalCodes":['14216'],
        "telecoms":[{'system':'PHONE',
                    'value':'7198675209'}],
        "addresses":[{"postalCode":'14216', 
                      "city":'Buffalo',
                      "state":'NY',
                      'lines':['680','Seneca St']
                     }],
#         "aliases":[{"type":'SOI',
#                     "value":'0123456789',
#                     "system":'2.16.840.1.113883.4.6'}],
        'sourceIdentifiers': []}
#body=json.dumps(body3)
o.create_organization(json=temp).json()

In [ ]:
p.delete_personnel('58973ab5-36da-4cb4-a6b4-1fbabbdfaf0c')
de05eb21-af4f-4ed5-a757-41fc9ec42e76

In [ ]:
from Gagnon, Patrick to everyone:    11:39 AM
https://cernercentral.com/system-accounts/f0c56327-1cc5-4dfa-8539-61aa72178542
from Barnes, Dave to everyone:    11:50 AM
joe
from Barnes, Dave to everyone:    11:50 AM
bearer
from Barnes, Dave to everyone:    11:50 AM
eyJhbGciOiJub25lIn0.eyJpc3MiOiJodHRwczpcL1wvYXV0aG9yaXphdGlvbi5jZXJuZXIuY29tXC8iLCJpYXQiOjE2Mjg4NjU4ODIsInVybjpjZXJuZXI6YXV0aG9yaXphdGlvbjpzeXN0ZW0tYWNjb3VudC1jcmVkZW50aWFsLXRva2VuOnZlcnNpb246MSI6eyJ2ZXIiOiIxLjAiLCJzZWNyZXQiOiJ0VzZ2dzRyeVMyU195VWZ6Y00yVy1ZU3RzR2tYM2VjMSIsImtleSI6ImYwYzU2MzI3LTFjYzUtNGRmYS04NTM5LTYxYWE3MjE3ODU0MiJ9LCJqdGkiOiI4MmM4MTg1Mi0zMmViLTRmZWQtYTg4Yi1iYTc1NThhNjIwNGIifQ.
from Gagnon, Patrick to everyone:    12:01 PM
# POST Testing
#import json
temp = {"name":"Rockcreek Clinic",
        "coverageAreaPostalCodes":['14216'],
        "telecoms":[{'system':'PHONE',
                    'value':'7198675209'}],
        "addresses":[{"postalCode":'14216', 
                      "city":'Buffalo',
                      "state":'NY',
                      'lines':['680','Seneca St']
                     }],
#         "aliases":[{"type":'SOI',
#                     "value":'0123456789',
#                     "system":'2.16.840.1.113883.4.6'}],
        'sourceIdentifiers': []}
#body=json.dumps(body3)
o.create_organization(json=temp).json()
from Gagnon, Patrick to everyone:    12:04 PM
# POST Testing Personnel
temp = {"name":{"given":'delete',"family":'me'}}
temp ={"birthDate":"1991-05-17",
        "gender":'MALE',
        "languages":['en'],
        "name":{"given":'delete',"family":'me',"middle":'z'},
        "addresses":[{"postalCode":'14210', 
                      "city":'Buffalo',
                      "state":'NY',
                      'lines':['608','Seneca St.']
                     }],
        "qualifications":[],
        "sourceIdentifiers":[]
}
#p.create_personnel(json=temp).json()
from Gagnon, Patrick to everyone:    12:05 PM
https://wny.personneltool.healtheintent.com/

In [8]:
org = pd.json_normalize(o.get_organization('e4dba7a2-6896-4878-8f32-560d4a0c0c38'))

In [11]:
reqResponse = o.get_organization('cbbc83fe-6994-4889-a915-499380903fad')

In [ ]:
reqResponse = o.get_organization_group('de05eb21-af4f-4ed5-a757-41fc9ec42e76')

In [12]:
print(reqResponse) 

{'items': [{'id': 'e4dba7a2-6896-4878-8f32-560d4a0c0c38', 'name': 'Akron Family Care', 'telecoms': [], 'addresses': [], 'aliases': [{'type': 'TAX', 'value': '161601272', 'system': '2.16.840.1.113883.4.2'}, {'type': 'SOI', 'value': '1619008869', 'system': '2.16.840.1.113883.4.6'}], 'sourceIdentifiers': [], 'coverageAreaPostalCodes': [], 'isManual': True, 'createdAt': '2019-11-19T14:46:53Z', 'updatedAt': '2021-05-11T16:06:13Z'}, {'id': 'cbbc83fe-6994-4889-a915-499380903fad', 'name': 'Alan M Antfleck Physician PLLC', 'telecoms': [], 'addresses': [], 'aliases': [{'type': 'SOI', 'value': '1366791998', 'system': '2.16.840.1.113883.4.6'}], 'sourceIdentifiers': [], 'coverageAreaPostalCodes': [], 'isManual': True, 'createdAt': '2021-03-02T14:59:25Z', 'updatedAt': '2021-03-02T14:59:25Z'}, {'id': '71522c48-58cd-467b-9b06-eaa22dcb94ee', 'name': 'Amandeep Pal MD PLLC', 'telecoms': [], 'addresses': [], 'aliases': [{'type': 'TAX', 'value': '472113619', 'system': '2.16.840.1.113883.4.2'}, {'type': 'SO

In [ ]:
organizations = pd.

In [9]:
org.head()

,items,firstLink,lastLink,nextLink,totalResults
0,[{'id': 'e4dba7a2-6896-4878-8f32-560d4a0c0c38'...,https://wny.api.us.healtheintent.com/organizat...,https://wny.api.us.healtheintent.com/organizat...,https://wny.api.us.healtheintent.com/organizat...,139


In [ ]:
org.shape

de05eb21-af4f-4ed5-a757-41fc9ec42e76

In [ ]:
from settings import *
import requests
import tarfile
import datetime as dt
import os

class Healtheintent(object):

    def __init__(self):

        self.uri = "https://wny.api.us.healtheintent.com/"
        self.headers = BEARER_HEADER
        self.headers_op = BEARER_HEADER_OP_API
        self.channel_id = None
        self.feed_id = None
        self.personnel_id = None
        self.organization_id = None
        self.personnel_group = None
        self.personnel_group_member_id = None
        self.location_id = None
        self.body = None

    def __send_request__(self, endpoint,
                         method = None, json = None, headers = None, params = None, data = None):
        if method == 'GET':
            return requests.get('{uri}{endpoint}'.format(uri=self.uri, endpoint=endpoint)
                                , headers=headers
                                , params=params).json()
        elif method == 'DELETE':
            return requests.delete('{uri}{endpoint}'.format(uri=self.uri, endpoint=endpoint)
                                   , headers=headers, params=params)
        elif method == 'POST':
            return requests.post('{uri}{endpoint}'.format(uri=self.uri, endpoint=endpoint),
                                   headers=headers, data=data, json=json)
        elif method == 'PUT':
            return requests.put('{uri}{endpoint}'.format(uri=self.uri, endpoint=endpoint),
                                   headers=headers, params=params,data=data,json=json)
        else:
            return requests.get('{uri}{endpoint}'.format(uri=self.uri, endpoint=endpoint)
                                , headers=headers, data=data, json=json)

#Data Syndication SubClass      
class DataSyndication(Healtheintent):

    def get_feeds(self):
        return self.__send_request__(endpoint='data-syndication/v1/feeds', method='GET', headers=self.headers)

    def get_channels(self):
        return self.__send_request__(endpoint='data-syndication/v1/channels', method='GET', headers=self.headers)

    def get_channel_status(self, channel_id = None):
        return self.__send_request__(endpoint='data-syndication/v1/channels/{channel_id}'.format(channel_id=channel_id),
                                     method='GET',
                                     headers=self.headers)

    def get_channel_deliveries(self, channel_id = None):
        return self.__send_request__(endpoint='data-syndication/v1/channels/{channel_id}/deliveries'.format(channel_id=channel_id),
                                     method='GET', headers=self.headers)

    def get_channel_downloads(self, delivery_id = None, destination_path = None):
        response = self.__send_request__(endpoint='data-syndication/v1/downloads/{delivery_id}'.format(delivery_id=delivery_id)
                                         , headers=self.headers)


        wd = os.getcwd()
        path = destination_path + '/' + dt.datetime.strftime(dt.datetime.today(), '%Y-%m-%d') + '.tar.gz'

        if response.status_code == 200:
            with open(path, 'wb') as i:
                i.write(response.content)
        i.close()

        temp = tarfile.open(name=path, mode='r:gz')
        os.chdir(destination_path)
        temp.extractall()
        temp.close()
        os.chdir(wd)


# Personnel Sub Class
class Personnel(Healtheintent):
    def get_personnel(self):
        return self.__send_request__(endpoint='personnel/v1/personnel', method='GET'
                                     , headers=self.headers_op)

    def create_personnel(self,data=None,json=None):
        return self.__send_request__(endpoint='personnel/v1/personnel'.format(data=data,json=json)
                                     , method='POST'
                                     , headers=self.headers_op
                                     , data=data
                                     , json=json)

    def get_personnel_status(self, personnel_id = None):
        return self.__send_request__(endpoint='personnel/v1/personnel/{personnel_id}'.format(personnel_id=personnel_id)
                                     , method='GET'
                                     , headers=self.headers_op)

    def update_personnel(self, personnel_id=None, data=None, json=None):
        return self.__send_request__(endpoint='personnel/v1/personnel/{personnel_id}'.format(personnel_id=personnel_id, data=data, json=json)
                                     , method='PUT', headers=self.headers_op, data=data, json=json)

    def delete_personnel(self, personnel_id = None):
        return self.__send_request__(endpoint='personnel/v1/personnel/{personnel_id}'.format(personnel_id=personnel_id)
                                     , method='DELETE'
                                     , headers=self.headers_op)

    
# Personnel Group Object
    def get_personnel_groups(self):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups'
                                     , method='GET'
                                     , headers=self.headers_op)

    def create_personnel_group(self, data=None, json=None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups'.format(data=data,json=json)
                                     , method='POST'
                                     , headers=self.headers_op)

    def get_personnel_group_status(self, personnel_group_id = None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups/{personnel_group_id}'.format(personnel_group_id=personnel_group_id)
                                     , method='GET'
                                     , headers=self.headers_op)

    def update_personnel_group(self, personnel_group_id = None, data=None, json=None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups/{personnel_group_id}'.format(personnel_group_id=personnel_group_id, data=data,json=json)
            , method='PUT'
            , headers=self.headers_op
            , data=data
            , json=json)

    def delete_personnel_group(self, personnel_group_id = None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups/{personnel_group_id}'.format(personnel_group_id=personnel_group_id)
                                     , method='DELETE'
                                     , headers=self.headers_op)

    def get_personnel_group_members(self, personnel_group_id = None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups/{personnel_group_id}/members'.format(personnel_group_id=personnel_group_id)
                                     , method='GET'
                                     , headers=self.headers_op)

    # Testing Required
    def add_personnel_group_members(self, personnel_group_id = None, personnel_group_member_id = None, data=None, json=None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups/{personnel_group_id}/members/{personnel_group_member_id}'
                                     , method='PUT'
                                     , headers=self.headers_op
                                    , data=data
                                    , json=json)

    def delete_personnel_group_members(self, personnel_group_id = None, personnel_group_member_id = None):
        return self.__send_request__(endpoint='personnel/v1/{personnel_group_id}/members/{personnel_group_member_id}'
                                     , method='DELETE'
                                     , headers=self.headers_op)


# Organization Sub Class Commands
class Organization(Healtheintent):
    
    def get_organization(self,params=None):
        return self.__send_request__(endpoint='organization/v1/organizations'.format(params=params)
                                     , method='GET'
                                     , headers=self.headers_op)
    
    def create_organization(self,data=None,json=None):
        return self.__send_request__(endpoint='organization/v1/organizations'.format(data=data, json=json)
                                     , method='POST'
                                     , headers=self.headers_op
                                     , data=data
                                     , json=json)
    
    def delete_organization(self, organization_id = None):
        return self.__send_request__(endpoint='organization/v1/organizations/{organization_id}'.format(organization_id=organization_id)
                                     , method='DELETE'
                                     , headers=self.headers_op)
    
    def update_organization(self, organization_id = None, data=None, json=None):
        return self.__send_request__(endpoint='organization/v1/organizations/{organization_id}'.format(organization_id=organization_id,data=data,json=json)
                                     , method='PUT'
                                     , headers=self.headers_op
                                    , data=data
                                    , json=json)
    
    
    # Location Object Commands
    def get_locations(self):
        return self.__send_request__(endpoint='organization/v1/locations'
                                     , method='GET'
                                     , headers=self.headers_op)

    def create_location(self,data=None,json=None):
        return self.__send_request__(endpoint='organization/v1/locations'.format(data=data, json=json)
                                     , method='POST'
                                     , headers=self.headers_op
                                     , data=data
                                     , json=json)
    
    def get_location_status(self, location_id = None):
        return self.__send_request__(endpoint='organization/v1/locations/{location_id}'.format(location_id=location_id)
                                     , method='GET'
                                     , headers=self.headers_op)

    def update_location(self, location_id = None, data=None, json= None):
        return self.__send_request__(endpoint='organization/v1/locations/{location_id}'.format(location_id=location_id,data=data,json=json)
                                     , method='PUT'
                                     , headers=self.headers_op
                                    , data=data
                                    , json=json)

    def delete_location(self, location_id = None):
        return self.__send_request__(endpoint='organization/v1/locations/{location_id}'.format(location_id=location_id)
                                     , method='DELETE'
                                     , headers=self.headers_op)